<a href="https://colab.research.google.com/github/Ninaad1007/Weight-Detection-using-Pills-Images/blob/main/PillWeight_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''!apt-get install unzip'''
'''import zipfile

# Path to the zip file in your Google Drive
zip_path = '/content/drive/MyDrive/pill_main.zip'

# Path to the destination folder where you want to extract the files
extract_path = '/content/drive/MyDrive'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)'''

In [ ]:
import pandas as pd
import os

path = '/content/drive/MyDrive/pill'
data = []
for filename in os.listdir(path):
    if filename.endswith('.JPG'):
        weight = filename.split('-')[0]
        data.append(weight)

df = pd.DataFrame(data, columns=['Weight'])
print(df)

   Weight
0   1.738
1   1.738
2   1.738
3   1.738
4   1.738
5   1.738
6   1.772
7   1.772
8   1.772
9   1.772
10  1.778
11  1.789
12  1.789
13  1.792
14  1.792
15  1.792
16  1.792
17  1.792
18  1.792
19  1.792
20  1.792
21  1.792
22  1.792
23  1.797
24  1.797
25  1.797
26  1.797
27  1.797
28  1.797
29  1.797
30  1.797
31  1.797
32  1.797
33  1.797
34  1.797
35  1.797
36  1.798
37    1.8
38    1.8
39    1.8
40    1.8
41    1.8
42    1.8
43    1.8
44    1.8


In [ ]:
#INVERTED ONE #RUNNING ONE
from tqdm import tqdm
import cv2
import os
import numpy as np

# Define the path to your image directory
path = 'drive/MyDrive/Pills dataset/pills detection project'

# Create empty lists to store your original and filtered images
org = []
images = []
segmented_images = []

# Loop through all the files in your image directory
for filename in tqdm(os.listdir(path)):
    # Load each image using OpenCV
    img = cv2.imread(os.path.join(path,filename))

    # Invert the colors of the image
    inverted_img = cv2.bitwise_not(img)

    # Convert the image to grayscale
    gray = cv2.cvtColor(inverted_img, cv2.COLOR_BGR2GRAY)

    # Apply a Gaussian blur with a 5x5 kernel
    blur = cv2.GaussianBlur(gray, (5, 5), 0)

    # Define a sharpening kernel
    kernel = np.array([[0,-1,0],[-1,5,-1],[0,-1,0]])

    # Apply the sharpening kernel
    sharpened = cv2.filter2D(blur, -1, kernel)

    # Perform watershed segmentation on the sharpened image
    thresh = cv2.threshold(sharpened, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)[1]
    kernel = np.ones((3,3),np.uint8)
    opening = cv2.morphologyEx(thresh,cv2.MORPH_OPEN,kernel, iterations = 2)
    sure_bg = cv2.dilate(opening,kernel,iterations=3)
    dist_transform = cv2.distanceTransform(opening,cv2.DIST_L2,5)
    ret, sure_fg = cv2.threshold(dist_transform,0.7*dist_transform.max(),255,0)
    sure_fg = np.uint8(sure_fg)
    unknown = cv2.subtract(sure_bg,sure_fg)
    ret, markers = cv2.connectedComponents(sure_fg)
    markers = markers+1
    markers[unknown==255] = 0
    markers = cv2.watershed(inverted_img,markers)
    inverted_img[markers == -1] = [255,0,0]
    mask = np.zeros(inverted_img.shape[:2], np.uint8)
    mask[markers != 1] = 255
    segmented_inverted = cv2.bitwise_and(sharpened, sharpened, mask=mask)

    # Invert the colors of the segmented image back
    segmented = cv2.bitwise_not(segmented_inverted)

    # Add the original, sharpened, and segmented images to their respective lists
    org.append(img)
    images.append(sharpened)
    segmented_images.append(segmented)

100%|██████████| 44/44 [00:17<00:00,  2.54it/s]


In [ ]:
import pickle

# Define the filename for the pickle file
filename = 'segmented_images.pkl'

# Store the list of segmented images in the pickle file
with open(filename, 'wb') as file:
    pickle.dump(segmented_images, file)

# Load the pickled list of segmented images
with open(filename, 'rb') as file:
    segmented_images = pickle.load(file)

In [ ]:
from google.colab.patches import cv2_imshow

img1 = segmented_images[15]
img3 = images[15]
img2 = org[15]
# Display your image
cv2_imshow(img1)
cv2_imshow(img2)
cv2_imshow(img3)
# Wait for a key press
cv2.waitKey(0)

# Close all windows
cv2.destroyAllWindows()

In [ ]:
import pickle
import numpy as np
import cv2
import pandas as pd
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.applications.mobilenet import preprocess_input

# Load the pickled list of segmented images
filename = '/content/drive/MyDrive/segmented_images.pkl'
with open(filename, 'rb') as file:
    segmented_images = pickle.load(file)

# Resize the images to the appropriate input size (224x224) for MobileNet and convert to RGB
resized_rgb_images = []
for img in segmented_images:
    resized_img = cv2.resize(img, (224, 224))
    if len(resized_img.shape) == 2:  # Grayscale image, convert to RGB
        resized_img = cv2.cvtColor(resized_img, cv2.COLOR_GRAY2RGB)
    resized_rgb_images.append(resized_img)

# Convert the list of resized images to a NumPy array
resized_rgb_images = np.array(resized_rgb_images)

# Preprocess the images for MobileNet
preprocessed_images = preprocess_input(resized_rgb_images)

# Load MobileNet without the classification layers
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Extract features from the images using MobileNet
features = base_model.predict(preprocessed_images)

# Save the features to a DataFrame
features_df = pd.DataFrame(features.reshape(features.shape[0], -1))

# Load the DataFrame containing weights (assuming you have another DataFrame named 'weights_df')
# Replace 'weights_df' with the actual DataFrame containing the 'Weight' column
# Make sure that 'weights_df' has the same number of rows and the correct order of weights as the images.

# Display the DataFrame as a table
print(features_df)
'''======================================================================================================='''
# If you want to display the DataFrame with the weights column, you can concatenate the DataFrames like this:
merged_df = pd.concat([features_df, df['Weight']], axis=1)
print(merged_df)

2/2 [==============================] - 1s 290ms/step
    0      1      2      3      4      5      6      7      8      9      ...  \
0     0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  ...   
1     0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  ...   
2     0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  ...   
3     0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  ...   
4     0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  ...   
5     0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  ...   
6     0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  ...   
7     0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  ...   
8     0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  ...   
9     0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  ...   
10    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0

In [ ]:
'''accuracy_list = []
psnr_list = []
mse_list = []
asnr_list = []

for i in range(len(org)):
    accuracy = cv2.norm(org[i], images[i], cv2.NORM_L2) / (org[i].shape[0] * org[i].shape[1])
    accuracy_list.append(accuracy)
    # Calculate PSNR
    psnr = cv2.PSNR(org[i], images[i])
    psnr_list.append(psnr)

    # Calculate MSE
    mse = np.square(np.subtract(org[i], images[i])).mean()
    mse_list.append(mse)

    # Calculate ASNR
    asnr = 10 * np.log10(np.mean(np.square(images[i])) / mse)
    asnr_list.append(asnr)

# Calculate PSNR, MSE, and ASNR
average_psnr = sum(psnr_list) / len(psnr_list)
average_mse = sum(mse_list) / len(mse_list)
average_asnr = sum(asnr_list) / len(asnr_list)

print("Average PSNR:", average_psnr)
print("Average MSE:", average_mse)
print("Average ASNR:", average_asnr)'''